# Boosting - ADABOOST


<!-- TOC START min:2 max:4 link:true asterisk:true update:true -->
* [What you will learn in this class](#what-you-will-learn-in-this-class)
* [Introduction](#introduction)
* [AdaBoost](#adaboost)
  * [Algorithm](#algorithm)
  * [Performance Condition](#performance-condition)
  * [Adaboost's overfitting](#adaboost--overfitting)
  * [Margins](#margins)
<!-- TOC END -->



## What you will learn in this class

Here we will define boosting, which allows to improve the performance of a simple supervised Machine Learning model by repeating the learning step several times in a row while taking into account prediction errors, and then combining the different models. This method uses the notion of "aggregating" that we have seen previously.

## Introduction

Boosting algorithms aim at transforming weak predictors into strong predictors, i.e. from models that are only partially correlated to the true distribution of the data to arrive at a new model that can approach this distribution as closely as desired.

While many boosting algorithms exist, the first major breakthrough in this field was the invention of the AdaBoost method by Schapire and Freund, which won the prestigious Gödel Prize for computer science in 2003.

Imagine you are preparing for an extremely difficult exam, you have read through the different chapters of your course once and you test your knowledge on former exam questions. How you score in this first mock exam will tell you what areas of the course you know well, and those for which you'll need a little more work. How will you plan your second round of studying based on your first attempt ? You will probably go through the content again and spend a little more time on what you had more difficulties with, and a little less time on the chapters you got right the first time around. Boosting follows exactly the same logic.

The general idea of the AdaBoost method is to successively build models that are sufficiently good predictors, by changing the weights assigned to observations, inflating the importance of those that cause errors and reducing the importance of those that are correctly classified by the model. In the end, the models are aggregated, and the weight of each model is a function of their training error.

## AdaBoost

### Algorithm

Certainly, the simplest way to introduce the Adaboost method is to describe the algorithm used in the form of pseudo-code. We consider here a modeling problem where we have observations $X_1,...,X_i,...,X_n$ which are vectors whose elements are the values of the explanatory variables, and a binary target variable $Y\;\in[-1,+1]$.

The algorithm is written as follows:

**Initialization :**

$D_{1}(i)=\frac{1}{n}\;pour\;{i}\in[1,n]$
At the beginning the Adaboost algorithm assigns equal weights to all observations.


**For** $t\;\in[1,T]$ **:**

* Train a model on the training data by weighting the observations with $D_1$

* We get a predictor that we'll note $h_t:\;X\rightarrow[-1,+1]$

* We calculate the weighted prediction error:

$\epsilon_t=\frac{1}{n}\sum_{i=1}^{n}1[h_t(X_i)\neq{Y_i}]\cdot{D_t(i)}$

* We calculate:

 $\alpha_t=\frac{1}{2}ln(\frac{1-\epsilon_t}{\epsilon_t})$, the weight of this model in the final aggregation

* We're updating the observation weights:

$D_{t+1}(i)=\frac{D_t(i)exp(-\alpha_t{Y_i}h_t(X_i))}{Z_t}$


Where $Z_t$ is simply the normalization factor so that

$\sum_{i=1}^{n}D_{t+1}(i)=1$


**Result:** the final model obtained from the AdaBoost algorithm is the following:


$H(X)=sign(\sum_{t=1}^{T}\alpha_t{h_t}(X))=sign(F(x))$


So we start by assigning the same weight to all observations, then we iteratively build prediction models, for example a logistic regression, at each iteration we modify the weights of the different observations, these weights will have an influence on the way the model parameters will be optimized by the model. The weights of the observations that were not well predicted increase, and the weights of the observations that were well predicted diminishes, which will force the next predictive model we will be training to pay more attention to the observations that the previous instance of the model had difficulties predicting. Ultimately, the model produced is a weighted voting aggregation of the different models according to their individual performance on the training data. Models that performed better will have more weight in the voting than models which performance was lower.



### Performance Condition

Intuitively, a binary classification model needs to gather three characteristics in order to be efficient and accurate:


* It must have been trained on a sufficient number of examples
* He must be able to describe the observations of the training base accurately, i.e. the training error must be small.
* It must be simple (an idea exists in statistics that in general, a simple model has a better chance of performing well on new data than a complex model, this idea is often cited as Ockham's razor).

These essential conditions need to be formalised mathematically in order to have practical sense. It is rather difficult to say precisely a priori how many observations are necessary to constitute a training set of sufficient size: it depends on the relative proportions of positive and negative observations and on the nature of the model that one wishes to use with the AdaBoost method. For example, we will probably need fewer observations to be able to correctly optimize a logistic regression (this does not mean that we will have good results, but at least we will achieve convergence of the parameters), and more observations if our binary classification model is more complex, such as a neural network.

The second performance criterion, commonly called the _weak learning condition_, corresponds to the idea that each binary classification model that makes up the final model has a training error that is smaller than pure chance. In mathematical terms this means that :

$\epsilon_t\;<\frac{1}{2}-\gamma$

Where $\gamma\;>0$.

Note that this specific criterion assumes that observations are equally distributed between the two classes. In general this criterion can written :

$\epsilon_t;< baseline - \gamma ; where ; \gamma;>0$



### Overfitting AdaBoost

Once these two conditions have been verified, the AdaBoost algorithm reduces the learning error to minimum level in relatively few iterations. However, as is often the case in statistics, a minimal training error often means overfitting ; the model sticks too closely to the training data. An example of that is shown in the figure below :


![sur_appr_adaboost](https://drive.google.com/uc?export=view&id=1-FEy8xiAoDBYtDwOjq8UOUv4x27-8kyY)


On the right you can see the theoretical behaviour of the model. It is expected to be observed if too many iterations are performed during learning. On the left, we present an example using the _Cleveland heart disease dataset_.



### Margins

We have seen in the previous example that the AdaBoost algorithm sometimes produces an overfitted model, i.e. it performs very well or even perfectly on the training data, but gives sub-optimal results on the validation data. However, it does not always fall into that trap. Indeed, there are cases where the algorithm produces a final model that predicts the validation data better and better when the number of iterations is increased. There is even a theoretical framework in which the AdaBoost algorithm provides a model where the error on the validation sample reaches an optimal level!

To explain this theoretical framework, the notion of margin is introduced. We have at our disposal a model consisting of aggregated weak learner models, the results of which are obtained using a weighted vote. However, we will not necessarily give the same confidence to the results obtained from the model for all observations, depending on the results of the vote. Just as in a real-world election, one is more likely to trust a decision that receives a large majority of votes rather than a small majority. Behind this idea of a strong or weak majority vote is the notion of margin. The margin is a value that fluctuates within the range *[-1.1]* which is defined as follows:


$Y_{i}F(x)=\sum_{t:Y_i=h_t(X_i)}\alpha_t\;-\sum_{t:Y_i\neq{h_t}(X_i)}\alpha_t$


It is therefore the difference between the cumulative weights of the models that predict correctly and the cumulative weights of the models that are wrong.

This concept of margin is essential because it can be shown that the model produced by the AdaBoost algorithm, and the set of models based on a majority vote of elementary models, have a generalization error (validation basis error) that does not depend on the number of iterations (the number of elementary models used to build the final predictor), but only on the margins obtained for the observations in the training sample. In other words, it does not matter how many models are used for the final vote: the higher the margins of observations in the training set, the smaller our validation error will be. In fact, if the addition of new models helps to increase the confidence margins, then we do not risk overfitting, on the contrary, the final model should perform better on the validation data.


![AdaBoost_perf](https://drive.google.com/uc?export=view&id=17DMihsYDzdH46VyuwJTZGxDPtYbWPiZY)


This example shows the performance of Adaboost applied to an OCR dataset for alphanumeric characters. On the right-hand side, we can see that despite a training error that reaches zero, the multiplication of iterations continues to lower the validation error. On the left, the cumulative margin distribution for five iterations is shown in dotted lines after five iterations, in dashes after one hundred iterations and in full after one thousand iterations. In situations where the algorithm allows to increase the margins of the training set observations, it is clear that the multiplication of iterations increases the predictive power of the final model.

## Gradient Boosting (XGBoost)

Another very popular boosting algorithm the gradient boosting, which works a little differently from Adaboost. Let $X$ be our training data and $y$ the target variable, the gradient boosting method will train a sequence of models noted $F_1, ..., F_m$ which goal is to predict $y$ as accurately as possible.
The algorithm starts by predicting $y$ using $F_1$ leading to :
$\hat{F_1}(X) = \hat{y}_1$ where $\hat{F_1}$ is the trained version of model $F_1$
This model is'nt necessarily super accurate at this point and we may want to add another model to predict $y$ better. Let's note this new predictor $f_1$, the new idea is to solve :
$F_2(X) = \hat{F_1}(X) + f_1(X) = y$
$f_1(X) = y - \hat{F_1}(X) = y - \hat{y}_1 = \epsilon_1$
The idea is therefore to train $f_1$ the new predictor in order for it predict the residuals $\epsilon_1$ from the first model, and not $y$ directly.
Iteratively the sequence of models is created with the following equation :
$F_{i+1} = \hat{F_{i}} + f_i$ 
where $f_i$ is trained on the residuals of $F_i$ noted $\epsilon_i$.

